Use base Python 3.12.2

# Model

In [16]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import ElectraModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm

device = "mps" if torch.backends.mps.is_available() else "cpu"

In [89]:
# 모델 정의
class KOTESentimentTagger(nn.Module):
    def __init__(self, pretrained_path):
        super().__init__()
        self.electra = ElectraModel.from_pretrained("beomi/KcELECTRA-base", revision='v2021')
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base", revision='v2021')
        self.intermediate_classifier = nn.Linear(self.electra.config.hidden_size, 44)
        self.final_classifier = nn.Linear(44, 3)  # 3-class 분류
        self.n_classes = 3

        # Pretrained 가중치 로드
        pretrained_state_dict = torch.load(pretrained_path, map_location=torch.device('cpu'))
        self.load_state_dict(pretrained_state_dict, strict=False)

    def forward(self, input_ids, attention_mask):
        output = self.electra(input_ids, attention_mask=attention_mask)
        output = output.last_hidden_state[:, 0, :]  # (batch_size, 768)
        intermediate_output = self.intermediate_classifier(output)  # (batch_size, 44)
        output = self.final_classifier(intermediate_output)  # (batch_size, 3)
        return output


# 모델 경로와 데이터 설정
trained_path = "/Users/jaesolshin/Documents/GitHub/youtube_dashboard/kote_finetuned_model.bin"
input_file = "/Users/jaesolshin/Documents/GitHub/youtube_dashboard/quota_sample_1000.csv"

# 모델 초기화 및 데이터 로드
model = KOTESentimentTagger(trained_path).to(device)
data = pd.read_csv(input_file)

# 테스트할 샘플 인덱스
sample_indices = [0, 20, 668, 790]

# 샘플 데이터 가져오기
sample_data = data.iloc[sample_indices]
comments = sample_data['comment'].tolist()
labels = sample_data['sentiment'].tolist()

# 토큰화 및 모델 입력 준비
encodings = model.tokenizer.batch_encode_plus(
    comments,
    add_special_tokens=True,
    max_length=512,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
    )

inputs = encodings['input_ids'].to(device)
attention_mask = encodings['attention_mask'].to(device)

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_49384/3869574122.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state_dict = torch.load(pretraine

In [51]:
# 추론
model.eval()
with torch.no_grad():
    outputs = model(inputs, attention_mask)
    softmaxs = nn.Softmax(dim=1)(outputs)
    preds = torch.argmax(softmaxs, dim=1)

# 출력 결과
print('inputs.shape', inputs.shape)
print('outputs.shape', outputs.shape)
print('labels', labels)
print('preds', preds.tolist())
print('softmaxs', softmaxs.tolist())

inputs.shape torch.Size([4, 512])
outputs.shape torch.Size([4, 3])
labels [0, 2, 0, 0]
preds [2, 2, 0, 0]
softmaxs [[0.014299365691840649, 0.019387319684028625, 0.966313362121582], [0.011146849021315575, 0.017465872690081596, 0.9713872671127319], [0.9796472787857056, 0.011131585575640202, 0.00922112911939621], [0.9549960494041443, 0.015010247007012367, 0.0299936942756176]]


In [32]:
# 샘플 데이터 예측결과 확인
def visualize_text_samples(sample_data, preds, softmaxs, model_name="KcELECTRA"):
    DESC = """
    Comment: {comment}
    True Label: {label}
    Prediction ({model_name}): {pred}
    Confidence Score (softmax): {softmax}
    """
    for comment, label, pred, softmax in zip(
        sample_data['comment'], sample_data['sentiment'], preds, softmaxs
    ):
        # Format the description
        desc = DESC.format(
            comment=comment,
            label=label,
            model_name=model_name,
            pred=pred,
            softmax=f"{max(softmax):.4f}"
        )
        print("="*50)
        print(desc)
    print("="*50)
# 텍스트 출력
visualize_text_samples(sample_data, preds, softmaxs, model_name="KcELECTRA")


    Comment: 뮤비 다이렉터와 코디... 컨셉 잡기 전 노래 들어본거 맞지?요즘 하이틴 감성 노래와 춤에다가 갑자기 중세시대 옷 그리고 군복.. 언밸러스 매력?ㅋㅋㅋ
    True Label: 0
    Prediction (KcELECTRA): 2
    Confidence Score (softmax): 0.9663
    

    Comment: 이 MV는 완벽합니다
    True Label: 2
    Prediction (KcELECTRA): 2
    Confidence Score (softmax): 0.9714
    

    Comment: 친북 친중하고 예수 찬양하는 놈들이 친일이랑 단월드를 욕할 자격은 있고? 내가 보기엔 둘 다 똑같은데? ㅋㅋㅋㅋㅋ 친북 친중 친일도 정신병자들이고 예수나 단월드나 다 눈에 보이지 않는 허상의 것이지 ㅋㅋㅋㅋ
    True Label: 0
    Prediction (KcELECTRA): 0
    Confidence Score (softmax): 0.9796
    

    Comment: 라이브는 절대 하지말고... 그러면 욕 안먹어... 립싱크와 퍼포먼스에 집중하도록...
    True Label: 0
    Prediction (KcELECTRA): 0
    Confidence Score (softmax): 0.9550
    


# XAI

In [ ]:
# !git clone --branch dev https://github.com/OpenXAIProject/pnpxai.git
# !pip install -e .
# !pip install gradio

%cd pnpxai

In [9]:
import sys
import site
sys.path.append(site.getsitepackages()[0])

import importlib
import pnpxai
importlib.reload(pnpxai)
%cd tutorials
%pwd

/opt/anaconda3/envs/pnpxai/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib created a temporary cache directory at /var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/matplotlib-f7nwwj4b because the default path (/Users/jaesolshin/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


/Users/jaesolshin/Documents/GitHub/youtube_dashboard/pnpxai/tutorials


## Detector and Recommender

In [45]:
dir(pnpxai)

['AutoExplanation',
 'AutoExplanationForImageClassification',
 'AutoExplanationForTSClassification',
 'AutoExplanationForTextClassification',
 'AutoExplanationForVisualQuestionAnswering',
 'Experiment',
 'XaiRecommender',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'core',
 'detect_model_architecture',
 'evaluator',
 'explainers',
 'messages',
 'utils']

In [46]:
from pnpxai import detect_model_architecture

detected_modules = detect_model_architecture(model)
print(detected_modules)

{<class 'pnpxai.core.detector.types.Linear'>, <class 'pnpxai.core.detector.types.Embedding'>}


In [43]:
from pnpxai import XaiRecommender
from pnpxai.core.modality import TextModality

recommender = XaiRecommender()
recommended = recommender.recommend(modality=TextModality(), model=model)
recommended.print_tabular()

----------------------  -------------------------------------------------------------------------------------------------------------------------
detected_architectures  ['Linear', 'Embedding']
explainers              ['Gradient', 'GradientXInput', 'IntegratedGradients', 'KernelShap', 'LRPUniformEpsilon', 'Lime', 'SmoothGrad', 'VarGrad']
----------------------  -------------------------------------------------------------------------------------------------------------------------


## Explainers

- A collection of SOTA XAI methods
- Easy interface to implement
- Various postprocess methods

### LRP

In [103]:
print("inputs type:", type(inputs))
print("inputs:", inputs)
print("attention_mask type:", type(attention_mask))
print("attention_mask:", attention_mask)
print("targets type:", type(preds))
print("targets:", preds)

inputs type: <class 'torch.Tensor'>
inputs: tensor([[    2,  1585,  4146,  ...,     0,     0,     0],
        [    2,  2741, 31966,  ...,     0,     0,     0],
        [    2, 11607, 10196,  ...,     0,     0,     0],
        [    2, 29036,  4082,  ...,     0,     0,     0]])
attention_mask type: <class 'torch.Tensor'>
attention_mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
targets type: <class 'torch.Tensor'>
targets: tensor([2, 2, 0, 0], device='mps:0')


In [123]:
from pnpxai.explainers import LRPEpsilonPlus

# LRP 초기화
lrp = LRPEpsilonPlus(model=model)

# LRP 속성 계산
attrs_lrp = lrp.attribute(inputs=inputs, targets=preds)
# model_inputs = {"input_ids": inputs, "attention_mask": attention_mask}
# attrs_lrp = lrp.attribute(inputs=model_inputs, targets=preds)

print("inputs.shape:", inputs.shape)
print("attrs_lrp.shape:", attrs_lrp.shape)

AttributeError: 'SequenceClassifierOutput' object has no attribute 'shape'

In [107]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from pnpxai.explainers import LRPEpsilonPlus

# 모델 및 토크나이저 로드
model_name = "distilbert-base-uncased-finetuned-sst-2-english"  # 기본 감정 분석 모델
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 감정 분석 대상 텍스트
texts = [
    "I love this product! It's amazing.",
    "I hate waiting in long lines.",
    "The movie was okay, not great but not bad either.",
]

# 토큰화 및 모델 입력 준비
encodings = tokenizer.batch_encode_plus(
    texts,
    add_special_tokens=True,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)

inputs = encodings["input_ids"].to(device)
attention_mask = encodings["attention_mask"].to(device)

# 모델 출력 및 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids=inputs, attention_mask=attention_mask)
    logits = outputs.logits  # 모델 출력
    preds = torch.argmax(logits, dim=1)  # 예측값

print("Predicted classes:", preds.tolist())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Predicted classes: [1, 0, 1]


In [124]:
from pnpxai.explainers import LRPEpsilonPlus

# LRP 초기화
lrp = LRPEpsilonPlus(model=model)

# LRP 속성 계산
attrs_lrp = lrp.attribute(inputs=inputs, targets=preds)
# model_inputs = {"input_ids": inputs, "attention_mask": attention_mask}
# attrs_lrp = lrp.attribute(inputs=model_inputs, targets=preds)

print("inputs.shape:", inputs.shape)
print("attrs_lrp.shape:", attrs_lrp.shape)

AttributeError: 'SequenceClassifierOutput' object has no attribute 'shape'